In [ ]:
import os
import sys
import numpy as np
import math
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from path_functions import smoothen_paths, check_if_avoids_obstacles
# from create_paths import mat_to_path
import matplotlib.pyplot as plt

import importlib.util
import sys
import data_manager as dm

from datetime import datetime
import time

Configure the evaluation run

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_grad_enabled(False)
GAN_TYPE = "AutoEncoder"   # The type of GAN being evaluated (See README.md for gan types, e.g. "AutoEncoder", "Pix2Pix")

# Checkpoint saving directory
# loc = os.getcwd()
loc = '/data'

DATASET = 'random_20_density'      # (String) The name of the dataset to load. Leave as None to use the same dataset that the model was trained on.
SUBSET = 'eval'     # The name of the subset to load from

RUN_NAME = 'fiery-glade-181'
STEP = 479

BATCH_SIZE = 1  # Must be 1 in order to get accurate estimation of prediction times
SMOOTH_VAL = 1

NUM_SAMPLES = 80     # the number of example outputs to generate
FIG_SCALE = 20      # the number of pixels per inch in the generated plots

Import the gan's class definitions

In [ ]:
# Make sure the cell only runs once
try:
    if MODEL_IMPORTED:
        print("Skipping cell")
except:
    config = dm.load_gan(RUN_NAME)

    from GANdefs import Generator
    gen = Generator(config['features_gen'], config['kernels_gen'], config['stride_gen'], config['padding_gen'], device=DEVICE)

    if DATASET == None:
        DATASET = config['dataset']
        SUBSET = config['subset']

MODEL_IMPORTED = True

In [ ]:
def path_to_tensor(path, map_dim):
    path_mat = torch.zeros(map_dim)

    # Make the path continuous
    for i in range(path.shape[0] - 1):
        x = int(path[i,1])
        x1 = int(path[i,1])
        x2 = int(path[i+1,1])

        y = int(path[i,0])
        y1 = int(path[i,0])
        y2 = int(path[i+1,0])

        if (x1 < x2):
            x_dir = 1
        else:
            x_dir = -1

        if (y1 < y2):
            y_dir = 1
        else:
            y_dir = -1

        # Determine y from x
        if x2-x1 != 0:
            m = (y2-y1)/(x2-x1)
            while x != x2:
                y = round(m*(x-x1) + y1)
                path_mat[y,x] = 1
                x += x_dir
        else:
            while x != x2:
                path_mat[y1,x] = 1
                x += x_dir


        x = int(path[i,1])
        x1 = int(path[i,1])
        x2 = int(path[i+1,1])

        y = int(path[i,0])
        y1 = int(path[i,0])
        y2 = int(path[i+1,0])

        # Determine x from y
        if y2-y1 != 0:
            m = (x2-x1)/(y2-y1)
            while y != y2:
                x = round(m*(y-y1) + x1)
                path_mat[y,x] = 1
                y += y_dir
        else:
            while y != y2:
                path_mat[y,x1] = 1
                y += y_dir
        
    path_mat[int(path[path.shape[0]-1,0]), int(path[path.shape[0]-1,1])] = 1     # Include the last point in the path

    return torch.tensor(path_mat)

Load the network(s)

In [ ]:
# Make sure the cell only runs once
try:
    if MODEL_LOADED:
        print("Skipping cell execution")
except:

    state = dm.load_checkpoint(RUN_NAME, STEP, loc)
    gen.load_state_dict(state['gen'])

MODEL_LOADED = True

Get the input set

In [ ]:
# Need to override __init__, __len__, __getitem__
# as per datasets requirement
class PathsDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, subset, device='cpu'):
        self.device = device
        self.paths = dm.load_input(dataset, subset) # Load all of the paths in the specified set

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = self.paths[idx]
        x = x.to(self.device)
        return x

In [ ]:
# Make sure the cell only runs once
try:
    if INPUT_LOADED:
        print("Skipping cell execution")
except:

    inputs = dm.load_input(DATASET, SUBSET) # load input gives us a tensor with 3 channels: real path, end points, SDF maps (in that order) 
    dataloader = DataLoader(inputs, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)

INPUT_LOADED = True

Get the resulting outputs

In [ ]:
pred_time = datetime(1,1,1,0,0)
start = time.time()
outputs = torch.Tensor()
for item_idx, item in enumerate(dataloader):
    item = item.to(DEVICE)

    # Generate some fake paths
    if GAN_TYPE == "AutoEncoder":
        noise = torch.rand_like(item[:,0:1,:,:])    # random noise matrix with same dimensions of the map (Note: may want try different noise distributions)
        noise = torch.cat((noise, item[:,1:,:,:]), axis=1)  # adding noise, then endpoints/line, then map to noise. (1: gives us all channels except truth/RRT* path)
    elif GAN_TYPE == "Pix2Pix":
        item = F.interpolate(item, size=(256,256))
        noise = item[:,1:,:,:]  # endpoints, map
    else:
        print(f"ERROR: '{GAN_TYPE}' is not a valid type of GAN")
        exit()

    t = datetime.now()
    fake = gen(noise)
    pred_time += datetime.now() - t

    out = torch.flip(item, [1])     # map, endpoints, truth
    out = torch.cat((out, fake[:,:1,:,:]), axis=1)  # map, endpoints, truth, generated (raw)

    out = out.to('cpu')
    outputs = torch.cat((outputs, out), axis=0)     # append the batch to the array of outputs
    if item_idx == NUM_SAMPLES:
        break

# Channels: map, endpoints, ground truth, generated (raw)
outputs = outputs[:NUM_SAMPLES,:,:,:]   # how many we want to display (from 0 to NUM_SAMPLES-1)
maps = outputs[:NUM_SAMPLES,:1,:,:]
maps = np.squeeze(maps)
maps[maps > 0] = 0  # converting from SDF to actual map so we can see if paths collide with obstacles
maps[maps < 0] = 1


end = time.time()
delta = end - start
print(delta)
print(NUM_SAMPLES)
print(f"took average {delta/NUM_SAMPLES} seconds per prediction")

avg_ms = pred_time.microsecond // NUM_SAMPLES
print(f"Average Computation Time: {avg_ms:d} ms")   # may encounter issue where avg time not represented correctly if total time >= 1s (could happen when NUM_SAMPLES is too large)

round outputs using Rachael's code

In [ ]:
# For each item, find start/end points using direct path (where direct path == 2)
# outputs = torch.tensor(outputs)
smooth = torch.zeros_like(outputs[:,-1:,:,:])
for i in range(outputs.shape[0]):
    # Find start/end
    ends = torch.nonzero(outputs[i,1,:,:] > 1)
    # ends = torch.flip(ends, [0])

    smooth[i] = path_to_tensor(smoothen_paths(outputs[i,-1:,:,:], ends[0], ends[1], smooth_val=SMOOTH_VAL, display=False)[0], outputs[i,-1,:,:].shape)[None,None,:,:]


outputs = torch.cat((outputs, smooth), axis=1)

# Channels: map, endpoints, ground truth, generated (raw), generated (smooth)

Format and save input, raw output, and rounded path

In [ ]:
from scipy.spatial import distance

outputs = np.asarray(outputs)

fig = plt.figure(figsize=(max(4, 4*outputs[0,0,:,:].shape[1]/FIG_SCALE),max(NUM_SAMPLES*(outputs[0,0,:,:].shape[1]/outputs[0,0,:,:].shape[0]), NUM_SAMPLES*outputs[0,0,:,:].shape[0]/FIG_SCALE)))

col_title = True

gan_total_length = 0 # variable to hold the total length of all LSTM generated paths
rrt_star_total_length = 0 # variable to hold the total length of all rrt* generated paths

for i in range(NUM_SAMPLES):
    gan_this_length = 0 # variable to hold length of this LSTM generated path
    rrt_star_this_length = 0 # variable to hold length of this rrt* generated path

    endpoints = np.squeeze(np.nonzero(outputs[i,1,:,:] == 2))

    # Show map
    sub = fig.add_subplot(NUM_SAMPLES,5,i*5+1)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(maps[i])
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Obstacles')

    # show real path
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+2)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,2,:,:], np.zeros_like(outputs[i,2,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Truth')
    # measure the distance of the RRT* training path
    rrt_star_xs = np.where(outputs[i,2,:,:]==1)[0]
    rrt_star_ys = np.where(outputs[i,2,:,:]==1)[1]
    path_len = len(rrt_star_xs)
    print(f"Path length: {path_len}")
    for itr in range (1,path_len):
        rrt_star_this_length = rrt_star_this_length + distance.euclidean((rrt_star_xs[itr], rrt_star_ys[itr]),(rrt_star_xs[itr-1], rrt_star_ys[itr-1]))
    print(f"distance of this RRT* path: {rrt_star_this_length}")
    rrt_star_total_length += rrt_star_this_length # incrament total path lengths (total length of all paths shown)

    # show raw gan output
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+3)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,3,:,:], np.zeros_like(outputs[i,3,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Generated')

    # Show smoothened gan output
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+4)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,-1,:,:], np.zeros_like(outputs[i,4,:,:]), maps[i]), axis=-1))
    xs = []
    ys = []
    f= open(f"paths/gan_{i}.txt","w+") # open a file for writing
    # for itr in outputs[i,4,:,:]:
    print(f"x: {np.where(outputs[i,4,:,:]==1)[0]}, y: {np.where(outputs[i,4,:,:]==1)[1]}")
    xs.append(np.where(outputs[i,4,:,:]==1)[0])
    ys.append(np.where(outputs[i,4,:,:]==1)[1])
    for itr in range(len(xs[0])):
        f.write(str(xs[0][itr]))
        f.write("\n")
        f.write(str(ys[0][itr]))
        f.write("\n")

    # measure the distance of the NDM-GAN training path
    gan_xs = np.where(outputs[i,4,:,:]==1)[0]
    gan_ys = np.where(outputs[i,4,:,:]==1)[1]
    path_len = len(gan_xs)
    print(f"Path length: {path_len}")
    for itr in range (1,path_len):
        gan_this_length = gan_this_length + distance.euclidean((gan_xs[itr], gan_ys[itr]),(gan_xs[itr-1], gan_ys[itr-1]))
    print(f"distance of this NDM-GAN path: {gan_this_length}")
    gan_total_length += gan_this_length # incrament total path lengths (total length of all paths shown)
        
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Smoothened')

    col_title = False

plt.show()

# calculate averge path lengths
print("******")
print(f'Average NMD-GAJ path length: {gan_total_length/NUM_SAMPLES}')
print(f'Average RRT* path length: {rrt_star_total_length/NUM_SAMPLES}')

In [ ]:
directory = f"{loc}/GAN_figs/{RUN_NAME}/{STEP}"#create figs directory if does not exist already
if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(NUM_SAMPLES):

    endpoints = np.squeeze(np.nonzero(outputs[i,1,:,:] == 2))

    # map_fig
    # sub = fig.add_subplot(NUM_SAMPLES,5,i*5+1)
    # sub.set_xticks([])
    # sub.set_yticks([])
    plt.figure()
    plt.imshow(maps[i])
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    plt.savefig(f"{loc}/GAN_figs/{RUN_NAME}/{STEP}/{i}_map.png")
    plt.close()

    # show real path
    # sub = fig.add_subplot(NUM_SAMPLES,4,i*4+2)
    # sub.set_xticks([])
    # sub.set_yticks([])
    # plt.imshow(np.stack((outputs[i,2,:,:], np.zeros_like(outputs[i,2,:,:]), maps[i]), axis=-1))
    # plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    # if col_title:
    #     plt.title('Truth')
    plt.figure()
    plt.imshow(np.stack((outputs[i,2,:,:], np.zeros_like(outputs[i,2,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    plt.savefig(f"{loc}/GAN_figs/{RUN_NAME}/{STEP}/{i}_real_path.png")
    plt.close()

    # # show raw gan output
    # sub = fig.add_subplot(NUM_SAMPLES,4,i*4+3)
    # sub.set_xticks([])
    # sub.set_yticks([])
    # plt.imshow(np.stack((outputs[i,3,:,:], np.zeros_like(outputs[i,3,:,:]), maps[i]), axis=-1))
    # plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    # if col_title:
    #     plt.title('Generated')
    plt.figure()
    plt.imshow(np.stack((outputs[i,3,:,:], np.zeros_like(outputs[i,3,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    plt.savefig(f"{loc}/GAN_figs/{RUN_NAME}/{STEP}/{i}_raw_gan_output.png")
    plt.close()

    # # Show smoothened gan output
    # sub = fig.add_subplot(NUM_SAMPLES,4,i*4+4)
    # sub.set_xticks([])
    # sub.set_yticks([])
    # plt.imshow(np.stack((outputs[i,-1,:,:], np.zeros_like(outputs[i,4,:,:]), maps[i]), axis=-1))
    # plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    # if col_title:
    #     plt.title('Smoothened')
    plt.figure()
    plt.imshow(np.stack((outputs[i,-1,:,:], np.zeros_like(outputs[i,4,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    plt.savefig(f"{loc}/GAN_figs/{RUN_NAME}/{STEP}/{i}_smoothend_gan_output.png")
    plt.close()

    # col_title = False

# Check how frequently the generated paths are continuous

The following cells contain code written by Rachael

Back to code which I've written

In [ ]:
try:
    if continuity >= 0:
        print(f"Predicted path continuity: {continuity:.2f}%")
except:
    CALC_CONTINUITY = True

    if CALC_CONTINUITY:
        # make predictions for entire dataset
        out = torch.Tensor()
        continuity = 0
        set_size = 0
        for item_idx, item in enumerate(dataloader):
            item = item.to(DEVICE)

            # Generate some fake paths
            if GAN_TYPE == "AutoEncoder":
                noise = torch.rand_like(item[:,0:1,:,:])    # Creates random matrix with same dimensions as map
                noise = torch.cat((noise, item[:,1:,:,:]), axis=1)  # noise, endpoints, map
            elif GAN_TYPE == "Pix2Pix":
                item = F.interpolate(item, size=(256,256)) # p2p works on 256x256 so resize what we have to this
                noise = item[:,1:,:,:]  # endpoints, map
            else:
                print(f"ERROR: '{GAN_TYPE}' is not a valid type of GAN")
                exit()
            
            fake = gen(noise)

            out = torch.flip(item, [1])     # map, endpoints, truth
            out = torch.cat((out[:,:-1,:,:], fake), axis=1)  # map, endpoints, generated (raw)

            # Converts maps back to binary occupancy grids
            maps = out[:,0,:,:]
            maps[maps > 0] = 0
            maps[maps < 0] = 1

            out = out.to('cpu')
            for i in range(out.shape[0]):   # go through current batch
                # Find start/end
                ends = torch.nonzero(out[i,1,:,:] > 1)  # getting the '2s' from the input's endpoint channel (note: see data_maneger.py>load_paths() )
                # ends = torch.flip(ends, [0])

                smooth = smoothen_paths(out[i,-1:,:,:], ends[0], ends[1], smooth_val=SMOOTH_VAL, display=False)[0]

                set_size += 1
                continuity += check_if_avoids_obstacles(smooth[:,1], smooth[:,0], maps[i,:,:])

        continuity *= 100
        continuity /= set_size

        print(f"Predicted path continuity: {continuity:.2f}%")